# 2 x $CO_2$

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
import importlib 
import pprint
import pymongo

from climatools.cliradlw import (setup, pipeline)
import climatools.cliradlw.runrecord as runrecord_clirad
import climatools.cliradlw.utils as utils_clirad

from climatools.atm import *

from IPython import display

In [3]:
client = pymongo.MongoClient(host='localhost', port=27017)
pd.set_option('max_colwidth', -1)

In [4]:
ATMPRO = {'mls': 294, 'saw': 257, 'trp': 300}
COMMITNUMBER = '1013f91'
MOL = 'co2'

In [14]:
atmcomp_olp = AtmComposition.cliradlw_nongreys()
atmcomp_olp.gasinbands = {band: gs for band, gs in atmcomp_olp.gasinbands.items() if MOL in gs}
atmcomp_olp.gasconcs[MOL] *= 2
atmcomps_olp = [AtmComposition(gasinbands={b:gs}, gasconcs=atmcomp_olp.gasconcs) 
                for b, gs in atmcomp_olp.gasinbands.items()]
parms_olp = [vars(p.to_cliradparam(commitnumber=COMMITNUMBER, atmpro=atmpro, tsfc=tsfc))
             for p in atmcomps_olp for atmpro, tsfc in ATMPRO.items()]

In [18]:
atmcomp_bft = AtmComposition.cliradlw_nongreys()
atmcomp_bft.gasinbands = {band: [MOL] for band, gs in atmcomp_bft.gasinbands.items() if MOL in gs}
atmcomp_bft.gasconcs[MOL] *= 2
atmcomps_bft = [AtmComposition(gasinbands={b:gs}, gasconcs=atmcomp_olp.gasconcs)
                for b, gs in atmcomp_bft.gasinbands.items()]
parms_bft = [vars(p.to_cliradparam(commitnumber=COMMITNUMBER, atmpro=atmpro, tsfc=tsfc))
             for p in atmcomps_bft for atmpro, tsfc in ATMPRO.items()]

In [25]:
def get_hrefs_clirad(param=None):
    atmpros = ('mls', 'saw', 'trp')
    hrefs = []
    for atmpro in atmpros:
        p = param.copy()
        p['atmpro'] = atmpro
        url = pipeline.nbviewer_url(param=p, setup=setup)
        s = '''<a href="{}">{}</a>'''
        s = s.format(url, atmpro)
        hrefs.append(s)
    s_hrefs = ' '.join(hrefs)
    return s_hrefs

def tb_hrefs_clirad():
    df = pd.DataFrame()
    for param in (parms_bft + parms_olp):
        hrefs = get_hrefs_clirad(param=param)
        bands = [str(b) for b in param['band']]
        df.loc['+'.join(param['molecule']), '+'.join(bands)] = hrefs
    df = df.fillna('')
    bmap = utils_clirad.mapband_new2old()
    lblnew_bands = [bmap[int(b)] for b in df.columns]
    columns = pd.MultiIndex.from_arrays([lblnew_bands, df.columns],
                                        names=['lblnew', 'clirad'])
    df.columns = columns
    return df

In [29]:
def script():
    df = tb_hrefs_clirad()
    s_html = df.to_html(escape=False)
    display.display(display.HTML(s_html))

In [27]:
script()

lblnew,3a,3b,3c,4,5,9
clirad,3,4,5,6,7,11
co2,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp
h2o+co2+n2o,mls saw trp,,,,,
h2o+co2,,mls saw trp,mls saw trp,mls saw trp,,mls saw trp
h2o+co2+o3,,,,,mls saw trp,


In [28]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')